# **IBM Applied Data Science Capstone Course by Coursera**

 * *Build a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name in Toronto.*
 * *Get the geographical coordinates of the neighborhoods in Toronto.*
 * *Explore and cluster the neighborhoods in Toronto (replicate the same analysis we did to New York City data).*
 
 -------
 ## **1. Import Library**


In [48]:
import numpy as np #to handle file in vectorize manner
import pandas as pd #for data analysis
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

import json #library to handle Json file

from geopy.geocoders import Nominatim # convert an adress into laditude and longitude

import requests #library to handle request

from bs4 import BeautifulSoup #library to parse HTML and XML documnents

from pandas.io.json import json_normalize #transform Json file into a pandas dataframe

#matplotlib and associated plotting module
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k mean from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Library imported.')

Library imported.


## 2. Scrap data from wikipedia page into DataFrame

In [77]:
# send get request
data= requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [78]:
#parse data from the html into a beautifulsoup object
soup=BeautifulSoup(data,'html.parser')

In [79]:
#create three list to store table data
postalCodeList=[]
boroughList=[]
neighborhoodList=[]

**Using beautifulSoup**

In [80]:
#find the table
soup.find('table').find_all('tr')

#find all rows of the table
soup.find('table').find_all('tr')

#for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells=row.find_all('td')

In [81]:
#append the data into the resoective lists
for row in soup.find('table').find_all('tr'):
    cells=row.find_all('td')
    if(len(cells)>0):
        postalCodeList.append(cells[0].text.rstrip('\n'))
        boroughList.append(cells[1].text.rstrip('\n'))
        neighborhoodList.append(cells[2].text.rstrip('\n')) #avoid new line in neighbourhood cell

In [82]:
#create a new dataFrame from the three lists

toronto_df=pd.DataFrame({'PostalCode':postalCodeList,'Borough':boroughList,'Neighborhood':neighborhoodList})
toronto_df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## 3. Drop the cell a borough that is "Not assigned"

In [83]:
#drop cells with borough that is not assigned
toronto_df_dropna=toronto_df[toronto_df.Borough !="Not assigned"].reset_index(drop=True)
toronto_df_dropna.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## 4. Group neighborhood in the same borough

In [86]:
#group neighborhood in the same borough
toronto_df_grouped=toronto_df_dropna.groupby(['PostalCode','Borough'],as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## 5. For Neighborhood='Not assigned', make the value the same as Borough

In [92]:
#for neighborhood='Not assigned', make the value the same as Borough

for index, row in toronto_df_grouped.iterrows():
    if row['Neighborhood']=='Not assigned':
        row['Neighborhood']=row['Borough']
        
toronto_df_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
